In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('./train.csv', encoding='gb2312', encoding_errors='ignore')
test_df = pd.read_csv('./test.csv', encoding='gb2312', encoding_errors='ignore')

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
train_df[:2]

In [ ]:
test_df[:2]

In [ ]:
train_df[-2:]

In [ ]:
test_df[-2:]

In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModel, GenerationConfig, AutoModelForCausalLM

model_path = "/root/share/new_models/Shanghai_AI_Laboratory/internlm2_5-20b-chat"
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
# 加载 label lora权重
# model = PeftModel.from_pretrained(model, './output/label_xfg').half()
# model = model.eval()
response, history = model.chat(tokenizer, "你好", history=[])
response

In [ ]:
def predict1(text):
    response, history = model.chat(tokenizer, f"以下是一个大模型完成的选择题。如果大模型的回答与参考答案一致，输出1；如果不一致，输出0。-->{text} ", history=[],
    temperature=0.01)
    return response

In [ ]:
def predict2(text):
    response, history = model.chat(tokenizer, f"以下是一个大模型完成的创作题目。按照流畅性评分标准给大模型创作打分(只取1分、2分、3分、4分、5分其一)。流畅性评分标准： 1分: 非常不流畅，不具备可读性：语法错误明显，难以理解；大量拼写错误和错别字，影响阅读；表达不清晰，难以捉摸要表达的意思。（平均每百字错误数 > 2.5个）; 
        2分: 具有可读性，但较不流畅：常见语法错误多，需花费一定时间理解；一些拼写错误和错别字，阅读中断；表达较为模糊，需用一些猜测才能明白含义。（平均每百字错误数 (2,2.5]个）;
        3分：基本流畅，存在少量语法错误，但影响较小：稍有拼写错误，但不影响阅读；主要意思表达清楚，但部分地方表述不够准确。（平均每百字错误数(1,2]个）;
        4分：较流畅，语法错误稀少，易读性较高：几乎无拼写错误，阅读顺畅；表达清晰、准确，容易理解。（平均每百字错误数(0.5,1]个）；
        5分：非常流畅，语法、拼写完美，阅读体验优秀：表达精炼、准确、得体；文句优美，行文连贯，思维严密。（平均每百字错误数[0,0.5]个）。-->{text}", history=[],
    temperature=0.1)
    return response

In [ ]:
def predict3(text):
    response, history = model.chat(tokenizer, f"以下是一个大模型完成的创作题目。按照规范性评分标准给大模型创作打分(只取1分、2分、3分、4分、5分其一)。规范性评分标准：
        1分: 创作内容离题，与提示语句要求不符，格式非常不规范：杂乱无章，句子结构混乱，缺乏逻辑。（平均每千字错误数 > 5个）; 
        2分: 创作内容与提示语句要求有一定契合但覆盖不全，格式较不规范：缺乏清晰的结构，但基本逻辑仍能找到。（平均每千字错误数(4,5]个）;
        3分：创作内容与提示语句要求基本契合但覆盖不全，格式一般规范：结构基本顺畅，逻辑较清晰。（平均每千字错误数 (2,4]个）;
        4分：创作内容与提示语句要求基本契合且基本覆盖，格式较规范：结构清晰，逻辑条理分明。（平均每千字错误数 (1,2]个）；
        5分：创作内容与提示语句要求完美契合，格式非常规范：结构严谨，逻辑清晰，层次分明。（平均每千字错误数 [0,1]个）-->{text}。", history=[],
    temperature=0.1)
    return response

In [ ]:
from tqdm import tqdm
label = []
for i in tqdm(range(len(test_df))):
    test_item = test_df.loc[i]
    if (llm_item['评判维度']=="选择题"):
        test_input = f"选择题:{test_item['创作要求']},评判内容:{test_item['待评判内容']}"
        label.append(int(predict1(test_input)))
    elif (llm_item['评判维度']=="流畅性"):
        test_input = f"大模型创作:{llm_item['待评判内容']}"
        label.append(int(predict2(test_input)))
    else:
        test_input = f"创作题目:{llm_item['创作要求']},大模型创作:{llm_item['待评判内容']}"
        label.append(int(predict3(test_input)))

In [ ]:
test_df['预测分数'] = label

In [ ]:
submit = test_df[['数据编号', '评判维度', '预测分数']]

In [ ]:
submit.to_csv('submit.csv', index=False)